In [3]:
import numpy as np
import torch
import pandas as pd
from sklearn.metrics import roc_auc_score
np.random.seed(2020)
torch.manual_seed(2020)
import pdb
import scipy.sparse as sps
import numpy as np
import torch
torch.manual_seed(2020)
from torch import nn
import torch.nn.functional as F
from math import sqrt

def sigmoid(x):
    return 1./(1. + np.exp(-x))

from dataset import load_data, load_features
from matrix_factorization import MF, MF_N_IPS, MF_N_DR_JL, MF_N_MRDR_JL
from baselines import MF, MF_IPS, MF_ASIPS, MF_SNIPS, MF_DR, MF_DR_JL, MF_MRDR_JL, MF_BaseModel
from models import MLP, MLP_exp, MLP_weibull, MLP_lognormal, MF_IPS_DF, MF_DR_JL_DF

from utils import gini_index, ndcg_func, get_user_wise_ctr, rating_mat_to_sample, binarize, shuffle, minU,recall_func, precision_func
from utils import ndcg_func_both, ndcg_func_feature, recall_func_both, recall_func_feature, generate_total_sample
mse_func = lambda x,y: np.mean((x-y)**2)
acc_func = lambda x,y: np.sum(x == y) / len(x)


dataset_name = "yahoo"

if dataset_name == "coat":
    train_mat, test_mat = load_data("coat")        
    x_train, y_train = rating_mat_to_sample(train_mat)
    x_test, y_test = rating_mat_to_sample(test_mat)
    num_user = train_mat.shape[0]
    num_item = train_mat.shape[1]

elif dataset_name == "yahoo":
    x_train, y_train, x_test, y_test = load_data("yahoo")
    x_train, y_train = shuffle(x_train, y_train)
    num_user = x_train[:,0].max() + 1
    num_item = x_train[:,1].max() + 1

print("# user: {}, # item: {}".format(num_user, num_item))
# binarize
y_train = binarize(y_train)
y_test = binarize(y_test)
n_train = x_train.shape[0]

train_user_ind = x_train[:, 0].astype('int')
train_item_ind = x_train[:, 1].astype('int')
test_user_ind = x_test[:, 0].astype('int')
test_item_ind = x_test[:, 1].astype('int')

# recover the complete matrix and retrieve the features

device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
mf = MF(num_user, num_item, batch_size=128)
mf.to(device)
mf.fit(x_train, y_train, 
    lr=0.05,
    lamb=1e-4,
    tol=1e-4)

x_train_x, u_emb_train, v_emb_train = mf.forward(x_train, True)
x_test_x, u_emb_test, v_emb_test = mf.forward(x_test, True)

# for the feature set of training dataset and test dataset
feature_train = torch.cat([u_emb_train, v_emb_train], axis = 1).detach().cpu()
feature_test = torch.cat([u_emb_test, v_emb_test], axis = 1).detach().cpu()

# the feature of user and item
user_W = mf.W(torch.LongTensor(np.arange(num_user) ).to(device) ).detach().cpu()
item_H = mf.H(torch.LongTensor(np.arange(num_item) ).to(device) ).detach().cpu()

L = 5
sigmaH = 0.1

num_feature = feature_train.shape[1]
identity_p = np.diag(np.ones(num_feature))
mean_p = np.zeros(num_feature)

W_d = np.random.multivariate_normal(mean_p, sigmaH**2*identity_p)
print(W_d)

y_train_mask = np.zeros_like(y_train)
e_train = np.zeros_like(y_train, dtype='float')
d_train = np.zeros_like(y_train, dtype='float') + 1e5

prod = 0
for i in range(n_train):

    ts_i = np.random.uniform(0, L)
    lambda_i = np.exp( np.dot(W_d, feature_train[i, :]) )
    d_i = np.random.exponential(lambda_i)
    e_i = L - ts_i
    if d_i <= e_i:
        y_train_mask[i] = y_train[i]
        d_train[i] = d_i
    else:
        if y_train[i] == 1:
            prod += 1
        y_train_mask[i] = 0
    
    e_train[i] = e_i

print('mask proportion (within the original y_train = 1 )')
print( prod/(sum(y_train) )*100 )

test_pred = mf.predict(x_test)
mse_mf = mse_func(y_test, test_pred)
auc_mf = roc_auc_score(y_test, test_pred)
print('prediction for base model')
print(auc_mf)

ips_idxs = np.arange(len(y_test))
np.random.shuffle(ips_idxs)
y_ips = y_test[ips_idxs[:int(0.05 * len(ips_idxs))]]

feature_test.to(device)

===>Load from yahoo data set<===
[train] num data: 311704
[test]  num data: 54000
# user: 15401, # item: 1001
[MF] epoch:11, xent:1643.3970547318459
[ 0.02036644  0.1544109   0.2018596  -0.01241452  0.11725286 -0.17158336
  0.04801624 -0.03370286]
mask proportion (within the original y_train = 1 )
19.47097722263042
prediction for base model
0.5759255535982502


tensor([[-0.0068, -0.0718, -0.0340,  ..., -0.0014,  0.0090,  0.0275],
        [-0.0068, -0.0718, -0.0340,  ...,  0.1090,  0.0541, -0.0475],
        [-0.0068, -0.0718, -0.0340,  ..., -0.0063,  0.0038,  0.3300],
        ...,
        [-0.1236,  0.0763, -0.1319,  ...,  0.0071, -0.1110,  0.1820],
        [-0.1236,  0.0763, -0.1319,  ..., -0.1814, -0.1245,  0.4052],
        [-0.1236,  0.0763, -0.1319,  ...,  0.0093, -0.0013,  0.0675]])

In [27]:
mf_ips_df = MF_IPS_DF(num_user, num_item, num_feature, batch_size=128)
mf_ips_df.to(device)
mf_ips_df.fit(x_train, y_train_mask, e_train, d_train, feature_train, y_ips,lr=0.015, lamb=1.5e-4, lamb1=8e-3, tol=1e-5)
test_pred = mf_ips_df.predict(x_test, feature_test)
mse = mse_func(y_test, test_pred)
auc = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func_both(mf_ips_df, x_test, y_test, feature_test)
recall_res = recall_func_both(mf_ips_df, x_test, y_test, feature_test)

print(auc)

[MF-IPS-DF] epoch:29, xent:63078.950622558594
0.6739248880454392


In [48]:
mf_dr_df = MF_DR_JL_DF(num_user, num_item, num_feature, batch_size=128)
mf_dr_df.to(device)
# 8e-3, 6e-3
# lambv= 
mf_dr_df.fit(x_train, y_train_mask, e_train, d_train, feature_train, user_W, item_H, y_ips, lr=0.015, lamb=6e-3, lambv=2e-1, tol=1e-5)
test_pred = mf_dr_df.predict(x_test)
mse = mse_func(y_test, test_pred)
auc = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func(mf_dr_df, x_test, y_test)
recall_res = recall_func(mf_dr_df, x_test, y_test)

print(auc)

[MF-DR-JL-DF] epoch:13, xent:5321667677.375
0.6229792266826


In [49]:
for lambv1 in [6e-2, 2e-2, 6e-3, 2e-3, 6e-4, 2e-4]:
    for lambv2 in [6e-1, 2e-1, 6e-2, 2e-2, 6e-3, 2e-3, 6e-4, 2e-4]:
        print("lamb:" + str(lambv1) + " lamb1:"+ str(lambv2))
        mf_dr_df = MF_DR_JL_DF(num_user, num_item, num_feature, batch_size=128)
        mf_dr_df.to(device)
        mf_dr_df.fit(x_train, y_train_mask, e_train, d_train, feature_train, user_W, item_H, y_ips, lr=0.015, lamb=6e-3, lambv=2e-1, tol=1e-5)
        test_pred = mf_dr_df.predict(x_test)
        mse = mse_func(y_test, test_pred)
        auc = roc_auc_score(y_test, test_pred)
        ndcg_res = ndcg_func(mf_dr_df, x_test, y_test)
        recall_res = recall_func(mf_dr_df, x_test, y_test)

        print(auc)


lamb:0.06 lamb1:0.6
[MF-DR-JL-DF] epoch:16, xent:5320759164.75
0.6314890746745259
lamb:0.06 lamb1:0.02
[MF-DR-JL-DF] epoch:14, xent:5320506767.25
0.6368539775665478
lamb:0.06 lamb1:0.06
[MF-DR-JL-DF] epoch:7, xent:3613717649.6875
0.6620493644337688
lamb:0.06 lamb1:0.02
[MF-DR-JL-DF] epoch:8, xent:3614514280.1875
0.6628358356400945
lamb:0.06 lamb1:0.006
[MF-DR-JL-DF] epoch:14, xent:5321146017.75
0.6268504273879778
lamb:0.06 lamb1:0.002
[MF-DR-JL-DF] epoch:17, xent:5320604919.875
0.6351912624933242
lamb:0.06 lamb1:0.0006
[MF-DR-JL-DF] epoch:7, xent:3614033244.9375
0.6619682909729676
lamb:0.06 lamb1:0.0002
[MF-DR-JL-DF] epoch:7, xent:3612605802.6875
0.6619554304072405
lamb:0.02 lamb1:0.6
[MF-DR-JL-DF] epoch:13, xent:5321084164.375
0.6375817525231275
lamb:0.02 lamb1:0.02
[MF-DR-JL-DF] epoch:12, xent:5321083722.875
0.6267116561356667
lamb:0.02 lamb1:0.06
[MF-DR-JL-DF] epoch:15, xent:5320715140.875
0.6302627956134673
lamb:0.02 lamb1:0.02
[MF-DR-JL-DF] epoch:8, xent:3617904600.375
0.655346770

In [50]:
for lrv in [0.03, 0.025, 0.02, 0.01, 0.005]:
    print('lr:'+str(lrv))
    mf_dr_df = MF_DR_JL_DF(num_user, num_item, num_feature, batch_size=128)
    mf_dr_df.to(device)
    mf_dr_df.fit(x_train, y_train_mask, e_train, d_train, feature_train, user_W, item_H, y_ips, lr=lrv, lamb=2e-4, lambv=2e-2, tol=1e-4)
    test_pred = mf_dr_df.predict(x_test)
    mse = mse_func(y_test, test_pred)
    auc = roc_auc_score(y_test, test_pred)
    ndcg_res = ndcg_func(mf_dr_df, x_test, y_test)
    recall_res = recall_func(mf_dr_df, x_test, y_test)

    print(auc)

lr:0.03
[MF-DR-JL-DF] epoch:16, xent:5315956128.25
0.5999784529439025
lr:0.025
[MF-DR-JL-DF] epoch:7, xent:3624968971.125
0.6426050710097433
lr:0.02
[MF-DR-JL-DF] epoch:13, xent:5315591209.5
0.6077962384328974
lr:0.01
[MF-DR-JL-DF] epoch:12, xent:5313943941.875
0.6391462990285286
lr:0.005
[MF-DR-JL-DF] epoch:19, xent:5312144373.625
0.6515511460789865


In [61]:
lambset = [(0.06, 0.06), (0.06, 6e-4), (6e-2, 2e-4), (6e-3, 2e-2), (6e-3, 6e-3), (2e-3, 6e-2), (6e-4, 6e-4), (6e-4, 2e-4)]

for lambv1, lambv2 in lambset:
    print('lambv1:'+str(lambv1)+ " lambv2:"+str(lambv2))
    mf_dr_df = MF_DR_JL_DF(num_user, num_item, num_feature, batch_size=128)
    mf_dr_df.to(device)
    mf_dr_df.fit(x_train, y_train_mask, e_train, d_train, feature_train, user_W, item_H, y_ips, lr=0.015, lamb=lambv1, lambv=lambv2, tol=1e-4)
    test_pred = mf_dr_df.predict(x_test)
    mse = mse_func(y_test, test_pred)
    auc = roc_auc_score(y_test, test_pred)
    ndcg_res = ndcg_func(mf_dr_df, x_test, y_test)
    recall_res = recall_func(mf_dr_df, x_test, y_test)

    print(auc)

lambv1:0.06 lambv2:0.06
[MF-DR-JL-DF] epoch:10, xent:5384668484.875
0.6344402832517986
lambv1:0.06 lambv2:0.0006
[MF-DR-JL-DF] epoch:7, xent:3673876509.625
0.66660915951617
lambv1:0.06 lambv2:0.0002
[MF-DR-JL-DF] epoch:7, xent:3692313520.3125
0.6645464083474775
lambv1:0.006 lambv2:0.02
[MF-DR-JL-DF] epoch:7, xent:3615371431.6875
0.6607907986802997
lambv1:0.006 lambv2:0.006
[MF-DR-JL-DF] epoch:8, xent:5320331388.25
0.6359330630440453
lambv1:0.002 lambv2:0.06
[MF-DR-JL-DF] epoch:8, xent:5317055648.125
0.6218884500544588
lambv1:0.0006 lambv2:0.0006
[MF-DR-JL-DF] epoch:10, xent:5313392855.75
0.6253327410213334
lambv1:0.0006 lambv2:0.0002
[MF-DR-JL-DF] epoch:8, xent:3585223869.4375
0.6455433968761205


In [65]:
# lambset = [(0.06, 6e-4), (6e-2, 2e-4), (6e-3, 2e-2)]
lambset = [(u, v) for u in ( 7.5e-3, 6e-3, 4.5e-3, 3e-3) for v in (3e-2, 2e-2, 1e-2, 8e-3)]
# the last one is of the best?

for lambv1, lambv2 in lambset:
    print('lambv1:'+str(lambv1)+ " lambv2:"+str(lambv2))
    mf_dr_df = MF_DR_JL_DF(num_user, num_item, num_feature, batch_size=256)
    mf_dr_df.to(device)
    mf_dr_df.fit(x_train, y_train_mask, e_train, d_train, feature_train, user_W, item_H, y_ips, lr=0.015, lamb=lambv1, lambv=lambv2, tol=1e-5)
    test_pred = mf_dr_df.predict(x_test)
    mse = mse_func(y_test, test_pred)
    auc = roc_auc_score(y_test, test_pred)
    ndcg_res = ndcg_func(mf_dr_df, x_test, y_test)
    recall_res = recall_func(mf_dr_df, x_test, y_test)

    print(auc)

lambv1:0.0075 lambv2:0.03
[MF-DR-JL-DF] epoch:12, xent:5322493422.75
0.6344357675631513
lambv1:0.0075 lambv2:0.02
[MF-DR-JL-DF] epoch:10, xent:5322897091.5
0.6306801396021391
lambv1:0.0075 lambv2:0.01
[MF-DR-JL-DF] epoch:11, xent:5322733460.125
0.6335998837467249
lambv1:0.0075 lambv2:0.008
[MF-DR-JL-DF] epoch:15, xent:5322362930.75
0.6344041827762896
lambv1:0.006 lambv2:0.03
[MF-DR-JL-DF] epoch:13, xent:5321130290.625
0.6292466828641818
lambv1:0.006 lambv2:0.02
[MF-DR-JL-DF] epoch:16, xent:5320541673.0
0.635516525909763
lambv1:0.006 lambv2:0.01
[MF-DR-JL-DF] epoch:11, xent:5320999086.25
0.6264828628489325
lambv1:0.006 lambv2:0.008
[MF-DR-JL-DF] epoch:15, xent:5321750199.875
0.625712944674424
lambv1:0.0045 lambv2:0.03
[MF-DR-JL-DF] epoch:7, xent:3609524166.625
0.6622179613763194
lambv1:0.0045 lambv2:0.02
[MF-DR-JL-DF] epoch:7, xent:3609731066.8125
0.6583781471141228
lambv1:0.0045 lambv2:0.01
[MF-DR-JL-DF] epoch:7, xent:3593270124.75
0.659372207127209
lambv1:0.0045 lambv2:0.008
[MF-DR-JL

In [66]:
# lambset = [(0.06, 6e-4), (6e-2, 2e-4), (6e-3, 2e-2)]
lambset = [(u, v) for u in ( 7.5e-3, 6e-3, 4.5e-3, 3e-3) for v in (3e-2, 2e-2, 1e-2, 8e-3)]
# the last one is of the best?

for lambv1, lambv2 in lambset:
    print('lambv1:'+str(lambv1)+ " lambv2:"+str(lambv2))
    mf_dr_df = MF_DR_JL_DF(num_user, num_item, num_feature, batch_size=256)
    mf_dr_df.to(device)
    mf_dr_df.fit(x_train, y_train_mask, e_train, d_train, feature_train, user_W, item_H, y_ips, lr=0.015, lamb=lambv1, lambv=lambv2, tol=1e-4)
    test_pred = mf_dr_df.predict(x_test)
    mse = mse_func(y_test, test_pred)
    auc = roc_auc_score(y_test, test_pred)
    ndcg_res = ndcg_func(mf_dr_df, x_test, y_test)
    recall_res = recall_func(mf_dr_df, x_test, y_test)

    print(auc)

lambv1:0.0075 lambv2:0.03
[MF-DR-JL-DF] epoch:8, xent:5322979131.5
0.6303783491011649
lambv1:0.0075 lambv2:0.02
[MF-DR-JL-DF] epoch:7, xent:3617225682.375
0.667123964390115
lambv1:0.0075 lambv2:0.01
[MF-DR-JL-DF] epoch:9, xent:5322828581.875
0.6282789380507579
lambv1:0.0075 lambv2:0.008
[MF-DR-JL-DF] epoch:8, xent:5322611986.625
0.6314106499684088
lambv1:0.006 lambv2:0.03
[MF-DR-JL-DF] epoch:8, xent:5321731537.375
0.6276359251697359
lambv1:0.006 lambv2:0.02
[MF-DR-JL-DF] epoch:8, xent:5321120277.25
0.6291633043802205
lambv1:0.006 lambv2:0.01
[MF-DR-JL-DF] epoch:8, xent:5321319959.125
0.636918671305544
lambv1:0.006 lambv2:0.008
[MF-DR-JL-DF] epoch:8, xent:5320899502.25
0.6309977802665481
lambv1:0.0045 lambv2:0.03
[MF-DR-JL-DF] epoch:8, xent:5318768994.875
0.6318186862466002
lambv1:0.0045 lambv2:0.02
[MF-DR-JL-DF] epoch:7, xent:3609608158.0625
0.6564440728523733
lambv1:0.0045 lambv2:0.01
[MF-DR-JL-DF] epoch:8, xent:5319435024.625
0.6236498776963761
lambv1:0.0045 lambv2:0.008
[MF-DR-JL-DF

In [68]:
mf_dr_df = MF_DR_JL_DF(num_user, num_item, num_feature, batch_size=256)
mf_dr_df.to(device)
mf_dr_df.fit(x_train, y_train_mask, e_train, d_train, feature_train, user_W, item_H, y_ips, lr=0.015, lamb=7.5e-4, lambv=2e-2, tol=1e-4)
test_pred = mf_dr_df.predict(x_test)
mse = mse_func(y_test, test_pred)
auc = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func(mf_dr_df, x_test, y_test)
recall_res = recall_func(mf_dr_df, x_test, y_test)

print(auc)

[MF-DR-JL-DF] epoch:8, xent:5313381131.875
0.6207252741316192


In [70]:
    mf_dr_df = MF_DR_JL_DF(num_user, num_item, num_feature, batch_size=256)
    mf_dr_df.to(device)
    mf_dr_df.fit(x_train, y_train_mask, e_train, d_train, feature_train, user_W, item_H, y_ips, lr=0.015, lamb=3e-4, lambv=0.01, tol=1e-4)
    test_pred = mf_dr_df.predict(x_test)
    mse = mse_func(y_test, test_pred)
    auc = roc_auc_score(y_test, test_pred)
    ndcg_res = ndcg_func(mf_dr_df, x_test, y_test)
    recall_res = recall_func(mf_dr_df, x_test, y_test)

    print(auc)

[MF-DR-JL-DF] epoch:8, xent:3569307049.6875
0.6474591946342666


In [73]:
mf_dr_df = MF_DR_JL_DF(num_user, num_item, num_feature, batch_size=256)
mf_dr_df.to(device)
mf_dr_df.fit(x_train, y_train_mask, e_train, d_train, feature_train, user_W, item_H, y_ips, lr=0.015, lamb=4.5e-4, lambv=0.03, tol=1e-4)
test_pred = mf_dr_df.predict(x_test)
mse = mse_func(y_test, test_pred)
auc = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func(mf_dr_df, x_test, y_test)
recall_res = recall_func(mf_dr_df, x_test, y_test)

print(auc)

mf_dr_df = MF_DR_JL_DF(num_user, num_item, num_feature, batch_size=256)
mf_dr_df.to(device)
mf_dr_df.fit(x_train, y_train_mask, e_train, d_train, feature_train, user_W, item_H, y_ips, lr=0.015, lamb=4.5e-4, lambv=0.02, tol=1e-4)
test_pred = mf_dr_df.predict(x_test)
mse = mse_func(y_test, test_pred)
auc = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func(mf_dr_df, x_test, y_test)
recall_res = recall_func(mf_dr_df, x_test, y_test)

print(auc)

mf_dr_df = MF_DR_JL_DF(num_user, num_item, num_feature, batch_size=256)
mf_dr_df.to(device)
mf_dr_df.fit(x_train, y_train_mask, e_train, d_train, feature_train, user_W, item_H, y_ips, lr=0.015, lamb=4.5e-4, lambv=0.01, tol=1e-4)
test_pred = mf_dr_df.predict(x_test)
mse = mse_func(y_test, test_pred)
auc = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func(mf_dr_df, x_test, y_test)
recall_res = recall_func(mf_dr_df, x_test, y_test)

print(auc)

[MF-DR-JL-DF] epoch:9, xent:5313960247.5
0.6188618699157811
[MF-DR-JL-DF] epoch:9, xent:5314617508.875
0.6221679365197161
[MF-DR-JL-DF] epoch:9, xent:3584543424.1875
0.6461446622637412


In [74]:
mlp = MLP(num_feature, batch_size=128)
mlp.to(device)
mlp.fit(x_train, y_train_mask, feature_train, lr=0.05, lamb=1e-4,tol=1e-5)
test_pred = mlp.predict(feature_test)
mse = mse_func(y_test, test_pred)
auc = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func_feature(mlp, x_test, y_test, feature_test)
recall_res = recall_func_feature(mlp, x_test, y_test, feature_test)

print(auc)

[MLP] epoch:13, xent:1631.1103895306587
0.5725012895661005


In [75]:
mlp_exp = MLP_exp(num_feature, batch_size=128)
mlp_exp.to(device)
mlp_exp.fit(x_train, y_train_mask, e_train, d_train, feature_train, lr=0.05, lamb=1e-4,tol=1e-5)
test_pred = mlp_exp.predict(feature_test)
mse = mse_func(y_test, test_pred)
auc = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func_feature(mlp_exp, x_test, y_test, feature_test)
recall_res = recall_func_feature(mlp_exp, x_test, y_test, feature_test)

print(auc)

[MLP_exp] epoch:11, xent:2128.7732273340225
0.5997989316935926


In [76]:
mlp_weibull = MLP_weibull(num_feature, batch_size=128)
mlp_weibull.to(device)
mlp_weibull.fit(x_train, y_train_mask, e_train, d_train, feature_train, lr=0.05, lamb=1e-4,tol=1e-5)
test_pred = mlp_weibull.predict(feature_test)
mse = mse_func(y_test, test_pred)
auc = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func_feature(mlp_weibull, x_test, y_test, feature_test)
recall_res = recall_func_feature(mlp_weibull, x_test, y_test, feature_test)

print(auc)

[MLP_weibull] epoch:14, xent:2131.2197816967964
0.5994565596904647


In [77]:
mlp_lognormal = MLP_lognormal(num_feature, batch_size=128)
mlp_lognormal.to(device)
mlp_lognormal.fit(x_train, y_train_mask, e_train, d_train, feature_train, lr=0.05, lamb=1e-4,tol=1e-5)
test_pred = mlp_lognormal.predict(feature_test)
mse = mse_func(y_test, test_pred)
auc = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func_feature(mlp_lognormal, x_test, y_test, feature_test)
recall_res = recall_func_feature(mlp_lognormal, x_test, y_test, feature_test)

print(auc)

[MLP_lognormal] epoch:13, xent:2216.64629137516
0.5982261105979498


In [86]:
mf = MF(num_user, num_item, batch_size=128)
mf.to(device)
mf.fit(x_train, y_train_mask, lr=0.05, lamb=3e-4,tol=1e-4)
test_pred = mf.predict(x_test)
mse = mse_func(y_test, test_pred)
auc = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func(mf, x_test, y_test)
recall_res = recall_func(mf, x_test, y_test)

print(auc)

[MF] epoch:8, xent:1687.8201692700386
0.501736095495157


In [87]:
mf_ips = MF_IPS(num_user, num_item, batch_size=128)
mf_ips.to(device)
mf_ips.fit(x_train, y_train_mask, y_ips, lr=0.05, lamb=8e-4,tol=1e-5)
test_pred = mf_ips.predict(x_test)
mse = mse_func(y_test, test_pred)
auc = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func(mf_ips, x_test, y_test)
recall_res = recall_func(mf_ips, x_test, y_test)

print(auc)

[MF-IPS] epoch:20, xent:65258.30628013611
0.6042178613610372


In [91]:
mf_snips = MF_SNIPS(num_user, num_item, batch_size=128)
mf_snips.to(device)
mf_snips.fit(x_train, y_train_mask, y_ips, lr=0.05, lamb=1e-4,tol=1e-5)
test_pred = mf_snips.predict(x_test)
mse = mse_func(y_test, test_pred)
auc = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func(mf_snips, x_test, y_test)
recall_res = recall_func(mf_snips, x_test, y_test)

print(auc)

[MF-SNIPS] epoch:20, xent:1580.0672101974487
0.5612743132018367


In [111]:
mf_asips = MF_ASIPS(num_user, num_item, batch_size=128)
mf_asips.to(device)
mf_asips.fit(x_train, y_train_mask, y_ips, tao=1e-2, lr=0.05, lamb=5e-5,tol=1e-5,batch_size=128,stop=5)
test_pred = mf_asips.predict(x_test)
mse = mse_func(y_test, test_pred)
auc = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func(mf_asips, x_test, y_test)
recall_res = recall_func(mf_asips, x_test, y_test)

print(auc)

[MF-IPS-Pred1] epoch:23, xent:63851.45141410828
[MF-IPS-Pred2] epoch:22, xent:63880.79926490784
[MF-ASIPS] epoch:16, xent:1276.1306467056274
0.6064912525555356


In [157]:
mf_dr = MF_DR(num_user, num_item, batch_size=128)
mf_dr.to(device)
mf_dr.fit(x_train, y_train_mask, y_ips, lr=0.05, lamb=1e-6,tol=1e-5)
test_pred = mf_dr.predict(x_test)
mse = mse_func(y_test, test_pred)
auc = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func(mf_dr, x_test, y_test)
recall_res = recall_func(mf_dr, x_test, y_test)

print(auc)

[MF-DR] epoch:12, xent:8011417.116943359
0.5899627181086522


In [121]:
mf_dr_jl = MF_DR_JL(num_user, num_item, batch_size=128)
mf_dr_jl.to(device)
# 0.03 1e-3
mf_dr_jl.fit(x_train, y_train_mask, y_ips, lr=0.03, lamb=1e-3,tol=1e-5)
test_pred = mf_dr_jl.predict(x_test)
mse = mse_func(y_test, test_pred)
auc = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func(mf_dr_jl, x_test, y_test)
recall_res = recall_func(mf_dr_jl, x_test, y_test)

print(auc)

[MF-DR-JL] epoch:14, xent:8014958.329833984
0.6188025892261613


In [126]:
mf_mrdr = MF_MRDR_JL(num_user, num_item, batch_size=128)
mf_mrdr.to(device)
mf_mrdr.fit(x_train, y_train_mask, y_ips, lr=0.05, lamb=1e-4,tol=1e-5)
test_pred = mf_mrdr.predict(x_test)
mse = mse_func(y_test, test_pred)
auc = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func(mf_mrdr, x_test, y_test)
recall_res = recall_func(mf_mrdr, x_test, y_test)

print(auc)

[MF-MRDR-JL] epoch:16, xent:8140195.770751953
0.6268744606724814


In [155]:
mf_dr_df = MF_DR_JL_DF(num_user, num_item, num_feature, batch_size=256)
mf_dr_df.to(device)
mf_dr_df.fit(x_train, y_train_mask, e_train, d_train, feature_train, user_W, item_H, y_ips, lr=0.015, lamb=3e-5, lambv=5e-3, tol=1e-4)
test_pred = mf_dr_df.predict(x_test)
mse = mse_func(y_test, test_pred)
auc = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func(mf_dr_df, x_test, y_test)
recall_res = recall_func(mf_dr_df, x_test, y_test)

print(auc)

[MF-DR-JL-DF] epoch:9, xent:5312941883.0
0.6276717454607252


In [ ]:
mf_dr_df = MF_DR_JL_DF(num_user, num_item, num_feature, batch_size=128)
mf_dr_df.to(device)
mf_dr_df.fit(x_train, y_train_mask, e_train, d_train, feature_train, user_W, item_H, y_ips, lr=0.03, lamb=3e-5, lambv=0.01, tol=1e-4)
test_pred = mf_dr_df.predict(x_test)
mse = mse_func(y_test, test_pred)
auc = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func(mf_dr_df, x_test, y_test)
recall_res = recall_func(mf_dr_df, x_test, y_test)

print(auc)

In [162]:
# for DR
mf_dr_acc = []
for repeat in np.arange(10):
    mf_dr = MF_DR(num_user, num_item, batch_size=128)
    mf_dr.to(device)
    mf_dr.fit(x_train, y_train_mask, y_ips, lr=0.03, lamb=5e-6,tol=1e-5)
    test_pred = mf_dr.predict(x_test)
    mse = mse_func(y_test, test_pred)
    auc = roc_auc_score(y_test, test_pred)
    ndcg_res = ndcg_func(mf_dr, x_test, y_test)
    recall_res = recall_func(mf_dr, x_test, y_test)

    print(auc)

    mf_dr_acc.append([ mse, auc, np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res['ndcg_10']), np.mean(recall_res['recall_5']), np.mean(recall_res['recall_10'])   ])

mf_dr_acc = np.array(mf_dr_acc)
mf_dr_mean = mf_dr_acc.mean(0)
mf_dr_sd = mf_dr_acc.std(0)

print("[MF_DR] test auc:", mf_dr_mean[1], ' sd: ', mf_dr_sd)

KeyboardInterrupt: 